In [ ]:
import data_analysis.data_processing as dp
import data_analysis.data_learning as dl

# reload module to bypass caching
import importlib
importlib.reload(dp)
importlib.reload(dl)

# treat it as a long path to avoid path length issues
test_window_path_home = dp.Path(r'\\?\C:\Users\jannis\Documents\HSP_IDS\Material\Aktuell\2025-02-17_11-14-33_192.168.1.0-normal_1\1554220324.748197-1554220354.748197')
test_window_path_remote = dp.Path(r'\\?\...')

test_window_home = dp.ThirtySecWindow(test_window_path_home)
#test_window_remote = da.ThirtySecWindow(test_window_path_remote)
dp.get_connection_features(test_window_home, True)

#da.connection_kmeans(test_window_home)
#dl.create_feature_vectors(test_window_home)


---- Processing connection 0 ----
---- Processing connection 1 ----
---- Processing connection 2 ----
---- Processing connection 3 ----
---- Processing connection 4 ----
---- Processing connection 5 ----
---- Processing connection 6 ----
---- Processing connection 7 ----
---- Processing connection 8 ----
---- Processing connection 9 ----
---- Processing connection 10 ----
{<data_analysis.data_processing.Host object at 0x000001DF2030FDD0>: {<data_analysis.data_processing.Connection object at 0x000001DF2030F0D0>:                0
0 -178721.865661
1   64078.077051
2   50747.332269
3   63896.456340, <data_analysis.data_processing.Connection object at 0x000001DF439D5AD0>:               0
0 -34501.478025
1 -30424.481473
2  21360.284495
3  23079.054971
4  20486.620032, <data_analysis.data_processing.Connection object at 0x000001DF439138D0>:               0
0 -35036.058183
1 -31054.780340
2  21714.539952
3  23570.044539
4  20806.254031, <data_analysis.data_processing.Connection object at 0x000